### Create the NPZ files of all electrodes

In [5]:
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np

st = study('respi')
sess = 'E1' #only one condition is sufficient for electrodes information
path = join(st.path, 'database/')
path2save = join(path, 'All_elecs_infos_npz_bipo/')
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ','MICP'] #respectively S0 S1 S2 S3 S4 S5
step = ''#FT

## Create a npz file with all sources informations
su_codes, s_xyz, s_channels,s_labels, s_elec = np.array([]),np.array([]), np.array([]), np.array([]),np.array([])
s_BA, s_AAL, s_AAL_RL, su_names, s_Mai, s_Mai_RL = np.array([]), np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
for i,su in enumerate(subjects):
    filename0 = path+su+'_'+sess+'_sigs_bipo3_phys'+step+'.npz'
    mat = np.load(filename0)
    #print(mat.files)
    xyz_all = mat['xyz']
    for elec in range(xyz_all.shape[0]):
        ### INFO electrodes for all subjects
        su_names = np.vstack((su_names,su)) if np.size(su_names) else su
        su_codes = np.vstack((su_codes,'S'+str(i))) if np.size(su_codes) else 'S'+str(i)
        xyz = xyz_all[elec]
        s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
        aal = mat['aal'][elec]
        s_AAL = np.vstack((s_AAL, aal)) if np.size(s_AAL) else aal
        aal_RL,BA = aal[:-4], mat['BA'][elec]
        s_AAL_RL = np.vstack((s_AAL_RL, aal_RL)) if np.size(s_AAL_RL) else aal_RL
        s_BA = np.vstack((s_BA, BA)) if np.size(s_BA) else BA
        label = mat['label'][elec]
        channel = mat['channel'][elec]               
        s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
        s_channels = np.vstack((s_channels, channel)) if np.size(s_channels) else channel
        s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
        if step == 'FT':
            Mai, Mai_RL = mat['Mai'][elec],mat['Mai_RL'][elec]
            s_Mai = np.vstack((s_Mai,Mai)) if np.size(s_Mai) else Mai
            s_Mai_RL = np.vstack((s_Mai_RL,Mai_RL)) if np.size(s_Mai_RL) else Mai_RL
            #print('Mai',mat['Mai'].shape,'Mai_RL',mat['Mai_RL'].shape,'BA',mat['BA'].shape)
su_names = np.squeeze(su_names)
su_codes = np.squeeze(su_codes)
s_labels = np.squeeze(s_labels)
s_AAL = np.squeeze(s_AAL)
s_AAL_RL = np.squeeze(s_AAL_RL)
s_BA = np.squeeze(s_BA)
s_channels = np.squeeze(s_channels)
s_elec = np.squeeze(s_elec)
if step == 'FT':
    s_Mai = np.squeeze(s_Mai)
    s_Mai_RL = np.squeeze(s_Mai_RL)
    #print ('after','aal',s_AAL.shape,'BA',s_BA.shape,'Mai',s_Mai.shape,'Mai_RL',s_Mai_RL.shape)
    su_coord_label = {'su_codes':su_codes,'su_names':su_names,'s_xyz':s_xyz,'s_labels':s_labels,
        's_channels':s_channels,'s_elec':s_elec,'s_Mai':s_Mai,'s_Mai_RL':s_Mai_RL,'s_aal':s_AAL,
        's_BA':s_BA, 's_aal_RL':s_AAL_RL}
else:
    su_coord_label = {'su_codes':su_codes,'su_names':su_names,'s_xyz':s_xyz,'s_labels':s_labels,
        's_channels':s_channels,'s_elec':s_elec,'s_aal':s_AAL, 's_BA':s_BA, 's_aal_RL':s_AAL_RL}
    #print ('after','aal',s_AAL.shape,'BA',s_BA.shape)
file_source = 'All_subjects_sources_sel_phys'+step+'.npz'
if not exists(path2save):
    makedirs(path2save)
np.savez(path2save+file_source,**su_coord_label)

-> respi loaded


### Plot the electrodes distribution

In [7]:
from brainpipe.system import study
from os.path import isfile, join
import numpy as np
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

step = ''
###############################################################################
st = study('respi')
path_npz = join(st.path, 'database/All_elecs_infos_npz_bipo/')
f_form = 'All_subjects_sources_sel_phys'+step+'.npz'
#f_form = 'All_subjects_sources_odor_'+conds[0]+'_sel_phys'+step+'.npz'
f_form = join(path_npz, f_form)
###############################################################################

#Create the df summarizing all elecs infos
mat = np.load(f_form)
subjects,su_codes, s_labels = mat['su_names'][:,np.newaxis],mat['su_codes'][:,np.newaxis], mat['s_labels'][:,np.newaxis]
s_elec, s_channels = mat['s_elec'][:,np.newaxis], mat['s_channels'][:,np.newaxis]
s_aal, s_BA = mat['s_aal'][:,np.newaxis], mat['s_BA'][:,np.newaxis]
aal_RL = np.array([s_aal[i][0][:-4][:] for i in range(s_aal.shape[0])])[:,np.newaxis]
print(subjects.shape, su_codes.shape,s_elec.shape, s_channels.shape, s_labels.shape,
      s_aal.shape,aal_RL.shape,s_BA.shape)
s_xyz = mat['s_xyz'].swapaxes(0,1)
s_x, s_y, s_z = s_xyz[0][:,np.newaxis], s_xyz[1][:,np.newaxis],s_xyz[2][:,np.newaxis]
print(s_x.shape, s_y.shape, s_z.shape)

if step == 'FT':
    s_Mai, s_Mai_RL = mat['s_Mai'][:,np.newaxis], mat['s_Mai_RL'][:,np.newaxis]
    print(s_Mai.shape, s_Mai_RL.shape)
    elec_data = np.concatenate((subjects,su_codes, s_elec, s_channels,s_labels,s_aal,
                            aal_RL,s_BA,s_Mai,s_Mai_RL,s_x, s_y,s_z), axis=1)
    df0 = pd.DataFrame(elec_data, columns=['subjects','su_codes','elecs','channels',
                    'labels','aal','aal_RL','BA','Mai','Mai_RL','x','y','z'])
else:
    elec_data = np.concatenate((subjects,su_codes, s_elec, s_channels,s_labels,s_aal,
                            aal_RL,s_BA,s_x, s_y,s_z), axis=1)
    df0 = pd.DataFrame(elec_data, columns=['subjects','su_codes','elecs','channels',
                    'labels','aal','aal_RL','BA','x','y','z'])

# Add information about the lobe of elecs
#dict_regions ={ 'ACC':'Frontal','IFG':'Frontal','MFG':'Frontal','SFG':'Frontal',
#                'Amg':'Olf','pPirT':'Olf','Amg-PirT':'Olf','Ins':'Olf','OFC':'Olf',
#                'HC':'MTL','PHG':'MTL','FuG':'Temporal','ITG':'Temporal',
#                'MTG':'Temporal','STG':'Temporal'}
#df0['region'] = df0['Mai_RL'].map(dict_regions)

dfname = '0_all_subjects_info_elecs'+step+'.csv'
df0.to_csv(path_npz+dfname)

-> respi loaded
(765, 1) (765, 1) (765, 1) (765, 1) (765, 1) (765, 1) (765, 1) (765, 1)
(765, 1) (765, 1) (765, 1)


In [5]:
### PLOTS BY ROIs AND SUBJECTS
colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 'S3':'mediumspringgreen',
          'S4':'yellow', 'S5':'darkorange', 'S6':'red'}

## total nb of electrodes by patient
elecs_su = df0[['su_codes','elecs']].groupby(['su_codes']).count()
print(elecs_su,'sel',elecs_su['elecs'])
fig = plt.figure()
plt.bar(range(len(elecs_su)), elecs_su['elecs'],
        color=[colors[i] for i,row in elecs_su.iterrows()])
plt.title('Total number of electrodes by subjects')
plt.ylabel('Nb of electrodes')
plt.xlabel('Subjects')
xticks_pos = np.arange(len(elecs_su))
plt.xticks(xticks_pos,list(elecs_su['elecs'].index.values))
plt.ylim(bottom=0, top=90)
plt.savefig(path_npz+'Total_elec_by_patient'+step+'.png')

          elecs
su_codes       
S0           60
S1           84
S2           60
S3           65
S4           52
S5           74 sel su_codes
S0    60
S1    84
S2    60
S3    65
S4    52
S5    74
Name: elecs, dtype: int64


<IPython.core.display.Javascript object>

In [4]:
from brainpipe.system import study
from os.path import isfile, join
import numpy as np
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
method='region' #'aal_RL','BA','Mai_RL'

st = study('Olfacto')
step = 'FT'
path_npz = join(st.path, 'database/Encoding_EpiPerf_LowHigh/All_elecs_infos_npz/')
df0 = pd.read_csv(path_npz+'0_all_subjects_info_elecs'+step+'.csv')
elecs_su_roi = df0.groupby(['su_codes',method]).count()['elecs'].unstack()
elecs_su_roi = elecs_su_roi.fillna(0)

#Plot nb of electrodes for each subjects
colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 'S3':'mediumspringgreen',
           'S4':'yellow', 'S5':'darkorange', 'S6':'red'}
fig = plt.figure(figsize=(7,7))
bottom = np.zeros(len(elecs_su_roi.columns))
for s in enumerate(elecs_su_roi.index):
    count = elecs_su_roi.iloc[elecs_su_roi.index.get_level_values('su_codes') == s[1]].values[0]
    xpts = range(len(count))
    print(s,count)
    plt.bar(xpts, count, bottom=bottom, color=colors[s[1]], label=s[1])
    bottom += count

plt.title('Nb of electrodes by ROI and Subject')
plt.ylabel('Nb of electrodes')
plt.xticks(xpts,list(elecs_su_roi.columns.values),rotation=90)
plt.tight_layout()
plt.ylim(bottom=0, top=150)
plt.legend(loc='best')
plt.savefig(path_npz+'Total_elec_by_patient_by_roi_'+method+step+'.png')
plt.savefig(path_npz+'Total_elec_by_patient_by_roi_'+method+step+'.pdf')

-> Olfacto loaded


<IPython.core.display.Javascript object>

(0, 'S0') [ 58.   0.   0.   0.]
(1, 'S1') [  4.  24.  18.  39.]
(2, 'S2') [ 17.   8.  11.  24.]
(3, 'S3') [ 41.   5.  16.   3.]
(4, 'S4') [  2.  18.  12.  20.]
(5, 'S5') [ 21.   7.  20.  26.]
(6, 'S6') [  0.  15.  15.  32.]


### Remove electrodes outside temporal-frontal regions

In [6]:
from os.path import join 
from brainpipe.system import study
import pandas as pd
import numpy as np

###################################################################################
st = study('Olfacto')
pathfiles = join(st.path, 'database/Retrieval_By_Odor/')
path_npz = join(pathfiles, 'All_elecs_infos_npz/')
###################################################################################
method = 'Mai_RL'
step = ''
df = pd.read_csv(path_npz+'0_all_subjects_info_elecs'+step+'.csv')
labels = df[method].unique()
to_keep = ['SFG','ACC','MFG','Amg','MTG','HC','PHG','FuG','ITG','Ins','STG',
           'pPirT','OFC','IFG','Amg-PirT']

files = [k for k in st.search('_sel_phys.npz', folder=('database/Retrieval_By_Odor/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    idx = np.where([mat[method][i] in to_keep for i in range(len(mat[method]))])
    print(fi[:4],idx[0].shape, idx,mat[method][idx])
    kwargs = {}
    kwargs['labels'], kwargs['x'], kwargs['aal'] = mat['labels'][idx], mat['x'][idx], mat['aal'][idx]
    kwargs['xyz'], kwargs['channels'], kwargs['sf'] = mat['xyz'][idx], mat['channels'][idx], mat['sf']
    kwargs['Mai'], kwargs['BA'], kwargs['Mai_RL'] = mat['Mai'][idx], mat['BA'][idx], mat['Mai_RL'][idx]
    np.savez(pathfiles+fi.replace('phys.npz','physFT.npz'),**kwargs)
    del mat

-> Olfacto loaded
CHAF (60,) (array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 63, 64, 65, 66, 67, 68, 69, 70, 71, 76, 77,
       78, 79, 80, 81, 82, 83, 91, 92, 93]),) ['SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'SFG' 'SFG' 'SFG' 'SFG' 'ACC' 'ACC' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'SFG' 'SFG' 'SFG' 'ACC' 'ACC' 'ACC' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'SFG' 'SFG' 'SFG' 'SFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG'
 'MFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'MFG' 'MFG' 'MFG']
CHAF (60,) (array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 63, 64, 65, 66, 67, 68, 69, 70, 71, 76, 77,
       78, 79, 80, 81, 82, 83, 91, 92, 93]),) ['SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' '

In [12]:
#Check for data dimensions
from brainpipe.system import study
import os
import numpy as np

st = study('respi')
path_data = os.path.join(st.path,'cycles_by_cond/')
files = os.listdir(path_data)
#print(files)
for fi in files:
    if fi.endswith('odor0.npy'):
        loadname = path_data+fi
        mat = np.load(loadname)
#         x = np.load(loadname)['x']
#         print (fi, x.shape, mat.files)
        print (fi, mat.shape)
        0/0

-> respi loaded
CHAF_E1_odor0.npy (113,)


ZeroDivisionError: division by zero